<div style=" background-color: RGB(0,114,200);" >
<h1 style="margin: auto; padding: 7px 0; color:#fff; text-align: center">PROJET 5</h1>
<h2 style="margin: auto; padding: 15px 0; color:#fff; text-align: center">BottleNeck - Optimiser la gestion des données d'une boutique
</h2>
</div>

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup


In [2]:
erp = pd.read_excel("../Source/Fichier_erp.xlsx")
web = pd.read_excel("../Source/Fichier_web.xlsx")
liaison = pd.read_excel("../Source/fichier_liaison.xlsx")

/Users/ishemzerzour/Desktop/My Program Files/Projet_4-BottleNeck/venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/Users/ishemzerzour/Desktop/My Program Files/Projet_4-BottleNeck/venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/Users/ishemzerzour/Desktop/My Program Files/Projet_4-BottleNeck/venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


<div style="background-color: RGB(0,150,250);" >
<h2 style="margin: auto; padding: 20px; color:#fff; ">Etape 1 - Exploration du fichier ERP (Référence produit)</h2>
</div>

In [3]:
erp.head()

,product_id,onsale_web,price,stock_quantity,stock_status
0,3847,1,24.2,0,outofstock
1,3849,1,34.3,0,outofstock
2,3850,1,20.8,0,outofstock
3,4032,1,14.1,0,outofstock
4,4039,1,46.0,0,outofstock


In [4]:
erp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 825 entries, 0 to 824
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   product_id      825 non-null    int64  
 1   onsale_web      825 non-null    int64  
 2   price           825 non-null    float64
 3   stock_quantity  825 non-null    int64  
 4   stock_status    825 non-null    object 
dtypes: float64(1), int64(3), object(1)
memory usage: 32.4+ KB


In [5]:
erp.columns

Index(['product_id', 'onsale_web', 'price', 'stock_quantity', 'stock_status'], dtype='object')

In [27]:
#product_id = Unique ? OUI 
print(erp['product_id'].nunique())
print('Les id produits sont uniques')
print(erp['stock_status'].nunique())
print('Les stock_status produits sont boolean')

825
Les id produits sont uniques
2
Les stock_status produits sont uniques


In [7]:
#onsale_web = Produits mis en ligne ou non ? Boolean
erp['onsale_web'].nunique() 

2

In [8]:
erp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 825 entries, 0 to 824
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   product_id      825 non-null    int64  
 1   onsale_web      825 non-null    int64  
 2   price           825 non-null    float64
 3   stock_quantity  825 non-null    int64  
 4   stock_status    825 non-null    object 
dtypes: float64(1), int64(3), object(1)
memory usage: 32.4+ KB


In [9]:
#stock_quantity = Beaucoup de 0 (à vérifier)
erp.describe()

,product_id,onsale_web,price,stock_quantity
count,825.000000,825.000000,825.000000,825.000000
mean,5162.597576,0.869091,32.384000,26.583030
std,902.644635,0.337506,26.832271,45.875948
min,3847.000000,0.000000,-8.000000,-1.000000
25%,4348.000000,1.000000,14.600000,1.000000
50%,4907.000000,1.000000,24.400000,11.000000
75%,5805.000000,1.000000,42.000000,34.000000
max,7338.000000,1.000000,225.000000,578.000000


In [10]:
#Il y a des prix négatifs ? les enlever ? assigner les valeurs absolu (positif) ? 
print(erp.loc[erp['price'] <= 0])
print('Les produits référencé qui ont un prix négatif ne sont pas vendus en ligne')

     product_id  onsale_web  price  stock_quantity stock_status
469        5017           0   -8.0               0   outofstock
739        6594           0   -1.0             192      instock
Les produits référencé qui ont un prix négatif ne sont pas vendus en ligne


In [11]:
#Il y a des stock_quantity négatifs ? les enlever ? assigner les valeurs absolu (positif) ?
erp.loc[erp['stock_quantity'] < 0]

,product_id,onsale_web,price,stock_quantity,stock_status
449,4973,0,10.0,-1,outofstock
573,5700,1,44.5,-1,outofstock


In [12]:
#Il y a des prix et des stock qui me paraissent élever comparer à la grande majorité des produits ? à voir

In [13]:
#stock_status = beaucoup de outofstock ?

In [14]:
#aucune valeurs null

<div style="background-color: RGB(0,150,250);" >
<h2 style="margin: auto; padding: 20px; color:#fff; ">Etape 2 - Exploration du fichier Web (Produits commercialisés en ligne)</h2>
</div>

In [15]:
def clean_html_tags(text):
    if pd.isna(text):
        return text
    else:
        soup = BeautifulSoup(text, "html.parser")
        return soup.get_text()


In [16]:
web

,sku,virtual,downloadable,rating_count,average_rating,total_sales,tax_status,tax_class,post_author,post_date,...,post_name,post_modified,post_modified_gmt,post_content_filtered,post_parent,guid,menu_order,post_type,post_mime_type,comment_count
0,16004,0,0,0,0.0,5.0,NaN,NaN,2.0,2018-06-07 16:27:25,...,chateau-du-couvent-pomerol-2017,2020-08-25 18:35:02,2020-08-25 16:35:02,NaN,0.0,https://www.bottle-neck.fr/wp-content/uploads/...,0.0,attachment,image/jpeg,0.0
1,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15075,0,0,0,0.0,3.0,taxable,NaN,2.0,2018-02-14 15:39:43,...,pares-balta-penedes-indigena-2017,2020-08-20 15:35:02,2020-08-20 13:35:02,NaN,0.0,https://www.bottle-neck.fr/?post_type=product&...,0.0,product,NaN,0.0
3,16209,0,0,0,0.0,6.0,taxable,NaN,2.0,2018-02-14 17:15:31,...,maurel-cabardes-tradition-2017,2020-08-05 18:05:03,2020-08-05 16:05:03,NaN,0.0,https://www.bottle-neck.fr/?post_type=product&...,0.0,product,NaN,0.0
4,15763,0,0,0,0.0,1.0,NaN,NaN,2.0,2020-01-25 14:08:16,...,domaine-de-la-monardiere-vacqueyras-les-vieill...,2020-08-21 11:35:02,2020-08-21 09:35:02,NaN,0.0,https://www.bottle-neck.fr/wp-content/uploads/...,0.0,attachment,image/jpeg,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1508,12881,0,0,0,0.0,2.0,NaN,NaN,2.0,2019-03-28 15:25:14,...,montbourgeau-etoile-vin-jaune-2009,2019-12-30 10:30:01,2019-12-30 09:30:01,NaN,0.0,https://www.bottle-neck.fr/wp-content/uploads/...,0.0,attachment,image/jpeg,0.0
1509,15663,0,0,0,0.0,3.0,taxable,NaN,2.0,2018-02-27 10:27:01,...,chermette-domaine-du-vissoux-brouilly-pierreux...,2020-08-01 09:35:02,2020-08-01 07:35:02,NaN,0.0,https://www.bottle-neck.fr/?post_type=product&...,0.0,product,NaN,0.0
1510,15910,0,0,0,0.0,0.0,taxable,NaN,2.0,2019-03-28 10:59:43,...,thevenet-quintaine-vire-clesse-la-bongran-2015,2020-08-14 10:45:02,2020-08-14 08:45:02,NaN,0.0,https://www.bottle-neck.fr/?post_type=product&...,0.0,product,NaN,0.0
1511,38,0,0,0,0.0,38.0,NaN,NaN,2.0,2018-04-18 12:25:58,...,emile-boeckel-cremant-brut-blanc-de-blancs,2020-08-27 17:15:03,2020-08-27 15:15:03,NaN,0.0,https://www.bottle-neck.fr/wp-content/uploads/...,0.0,attachment,image/jpeg,0.0


In [17]:
web.shape

(1513, 28)

In [18]:
web.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1513 entries, 0 to 1512
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   sku                    1428 non-null   object        
 1   virtual                1513 non-null   int64         
 2   downloadable           1513 non-null   int64         
 3   rating_count           1513 non-null   int64         
 4   average_rating         1430 non-null   float64       
 5   total_sales            1430 non-null   float64       
 6   tax_status             716 non-null    object        
 7   tax_class              0 non-null      float64       
 8   post_author            1430 non-null   float64       
 9   post_date              1430 non-null   datetime64[ns]
 10  post_date_gmt          1430 non-null   datetime64[ns]
 11  post_content           0 non-null      float64       
 12  post_title             1430 non-null   object        
 13  pos

In [19]:
web.columns

Index(['sku', 'virtual', 'downloadable', 'rating_count', 'average_rating',
       'total_sales', 'tax_status', 'tax_class', 'post_author', 'post_date',
       'post_date_gmt', 'post_content', 'post_title', 'post_excerpt',
       'post_status', 'comment_status', 'ping_status', 'post_password',
       'post_name', 'post_modified', 'post_modified_gmt',
       'post_content_filtered', 'post_parent', 'guid', 'menu_order',
       'post_type', 'post_mime_type', 'comment_count'],
      dtype='object')

In [20]:
#supprimer les colonnes totalement vide: tax_class, post_content, post_password
web.drop(columns=["tax_class", "post_content", "post_password", "post_content_filtered"], inplace=True)
web.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1513 entries, 0 to 1512
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   sku                1428 non-null   object        
 1   virtual            1513 non-null   int64         
 2   downloadable       1513 non-null   int64         
 3   rating_count       1513 non-null   int64         
 4   average_rating     1430 non-null   float64       
 5   total_sales        1430 non-null   float64       
 6   tax_status         716 non-null    object        
 7   post_author        1430 non-null   float64       
 8   post_date          1430 non-null   datetime64[ns]
 9   post_date_gmt      1430 non-null   datetime64[ns]
 10  post_title         1430 non-null   object        
 11  post_excerpt       716 non-null    object        
 12  post_status        1430 non-null   object        
 13  comment_status     1430 non-null   object        
 14  ping_sta

In [21]:
#supprimer les lignes totalement vide


#Je ne peux pas car certainens colonnes sont pas des NaN mais des 0

In [22]:
#sku manquant, que faut-il faire pour ceux-la ? 

<div style="background-color: RGB(0,150,250);" >
<h2 style="margin: auto; padding: 20px; color:#fff; ">Etape 3 - Exploration du fichier Liaison</h2>
</div>

In [23]:
liaison

,product_id,id_web
0,3847,15298
1,3849,15296
2,3850,15300
3,4032,19814
4,4039,19815
...,...,...
820,7203,NaN
821,7204,NaN
822,7247,13127-1
823,7329,14680-1


In [24]:
liaison.loc[liaison['product_id'] == 6594]

,product_id,id_web
739,6594,NaN


In [25]:
#Renommer la colonne id_web en sku
liaison = liaison.rename(columns={'id_web': 'sku'})
liaison.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 825 entries, 0 to 824
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   product_id  825 non-null    int64 
 1   sku         734 non-null    object
dtypes: int64(1), object(1)
memory usage: 13.0+ KB


In [26]:
#supprimer les colonnes inutiles (aucune valeurs): Unnamed: 2, Unnamed: 3, Unnamed: 4
drop_col = ['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4']

liaison.drop(columns=drop_col, inplace=True)

liaison

KeyError: "['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'] not found in axis"

In [ ]:
#on va verifier si les product id sont unique
print(liaison['product_id'].nunique())
print("les product_id sont bien uniques")

825
les product_id sont bien uniques


In [ ]:
#on va sortir les lignes ou les sku sont vide
liaison[liaison.isnull().any(axis=1)]

,product_id,sku
19,4055,NaN
49,4090,NaN
50,4092,NaN
119,4195,NaN
131,4209,NaN
...,...,...
817,7196,NaN
818,7200,NaN
819,7201,NaN
820,7203,NaN


In [ ]:
#id_web c'est de l'objet ? convertir en int

<div style="background-color: RGB(0,150,250);" >
<h2 style="margin: auto; padding: 20px; color:#fff; ">Etape 4 - Analyse du fichier ERP avec Liaison</h2>
</div>

In [ ]:
#1 : Je merge ERP et liaison pour vérifier si les ref_sku manquant sont des produit commercialisé off_line

erp_liaison = pd.merge(erp, liaison, how="left", on="product_id")
erp_liaison

,product_id,onsale_web,price,stock_quantity,stock_status,sku
0,3847,1,24.2,0,outofstock,15298
1,3849,1,34.3,0,outofstock,15296
2,3850,1,20.8,0,outofstock,15300
3,4032,1,14.1,0,outofstock,19814
4,4039,1,46.0,0,outofstock,19815
...,...,...,...,...,...,...
820,7203,0,45.0,30,instock,NaN
821,7204,0,45.0,9,instock,NaN
822,7247,1,54.8,23,instock,13127-1
823,7329,0,26.5,14,instock,14680-1


In [ ]:
#1.2 J'affiche uniquement les données des produits référencer sur le site : "onsale_web" == 1

erp_liaison = erp_liaison.loc[erp_liaison['onsale_web'] == 1]
erp_liaison

,product_id,onsale_web,price,stock_quantity,stock_status,sku
0,3847,1,24.2,0,outofstock,15298
1,3849,1,34.3,0,outofstock,15296
2,3850,1,20.8,0,outofstock,15300
3,4032,1,14.1,0,outofstock,19814
4,4039,1,46.0,0,outofstock,19815
...,...,...,...,...,...,...
787,6930,1,8.4,83,instock,16135
792,7023,1,27.5,15,instock,15891
793,7025,1,69.0,2,instock,15887
822,7247,1,54.8,23,instock,13127-1


In [ ]:
#1.3 J'affiche les ligne qui ont un sku null (la ce serai un problème)
erp_liaison.info()


<class 'pandas.core.frame.DataFrame'>
Index: 717 entries, 0 to 824
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   product_id      717 non-null    int64  
 1   onsale_web      717 non-null    int64  
 2   price           717 non-null    float64
 3   stock_quantity  717 non-null    int64  
 4   stock_status    717 non-null    object 
 5   sku             714 non-null    object 
dtypes: float64(1), int64(3), object(2)
memory usage: 39.2+ KB


In [ ]:
#1.4 il y a 3 sku manquant, je vais afficher les lignes
erp_liaison[erp_liaison.isnull().any(axis=1)]

,product_id,onsale_web,price,stock_quantity,stock_status,sku
242,4594,1,144.0,0,outofstock,NaN
486,5070,1,84.7,0,outofstock,NaN
487,5075,1,43.3,0,outofstock,NaN


In [ ]:
erp_liaison

,product_id,onsale_web,price,stock_quantity,stock_status,sku
0,3847,1,24.2,0,outofstock,15298
1,3849,1,34.3,0,outofstock,15296
2,3850,1,20.8,0,outofstock,15300
3,4032,1,14.1,0,outofstock,19814
4,4039,1,46.0,0,outofstock,19815
...,...,...,...,...,...,...
787,6930,1,8.4,83,instock,16135
792,7023,1,27.5,15,instock,15891
793,7025,1,69.0,2,instock,15887
822,7247,1,54.8,23,instock,13127-1


In [ ]:
erp_liaison.to_numpy()

array([[3847, 1, 24.2, 0, 'outofstock', '15298'],
       [3849, 1, 34.3, 0, 'outofstock', '15296'],
       [3850, 1, 20.8, 0, 'outofstock', '15300'],
       ...,
       [7025, 1, 69.0, 2, 'instock', '15887'],
       [7247, 1, 54.8, 23, 'instock', '13127-1'],
       [7338, 1, 16.3, 45, 'instock', '16230']], dtype=object)

In [ ]:
erp_liaison.describe()

,product_id,onsale_web,price,stock_quantity
count,717.000000,717.0,717.000000,717.000000
mean,5032.057183,1.0,32.736541,28.641562
std,789.025959,0.0,28.132912,47.940822
min,3847.000000,1.0,5.200000,-1.000000
25%,4281.000000,1.0,14.100000,2.000000
50%,4797.000000,1.0,23.700000,12.000000
75%,5709.000000,1.0,42.600000,35.000000
max,7338.000000,1.0,225.000000,578.000000
